# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(appName='sga')

from pyspark.sql import SparkSession, Row
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-26 11:16:41,611 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [5]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window

In [7]:
import pandas as pd

clickstream_df = pd.read_csv('./clickstream.csv', sep='\t')

print(clickstream_df.head())

   user_id  session_id    event_type event_page   timestamp
0      562         507          page       main  1695584127
1      562         507         event       main  1695584134
2      562         507         event       main  1695584144
3      562         507         event       main  1695584147
4      562         507  wNaxLlerrorU       main  1695584154


In [8]:
spark_clickstream_df = spark.createDataFrame(clickstream_df)

spark_clickstream_df.createOrReplaceTempView('clickstream')

# Using SQL

In [9]:
routes_sql = spark.sql("""
WITH error_flagged AS (
    SELECT 
        user_id, 
        session_id, 
        event_page, 
        timestamp,
        MAX(CASE WHEN event_type LIKE '%error%' THEN 1 ELSE 0 END) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS error_flag
    FROM 
        clickstream
),

duplicate_flagged AS (
    SELECT 
        user_id, 
        session_id,
        event_page,
        timestamp,
        CASE WHEN LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) = event_page THEN 1 ELSE 0 END AS duplicate_flag
    FROM 
        error_flagged
    WHERE
        error_flag <> 1
),

all_routes AS (
    SELECT 
        user_id, 
        session_id,
        CONCAT_WS('-', COLLECT_LIST(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp)) AS route
    FROM 
        duplicate_flagged
    WHERE 
        duplicate_flag <> 1
),

routes_per_user_sess AS (
    SELECT
        user_id, 
        session_id,
        MAX(route) AS route
    FROM 
        all_routes
    GROUP BY
        user_id, session_id
)

SELECT 
    route,
    COUNT(*) as count
FROM 
    routes_per_user_sess 
GROUP BY
    route
ORDER BY 
    count DESC
LIMIT 30
""")

routes_sql.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|main-tariffs-archive|  103|
|  main-internet-news|  103|
|    main-rabota-main|   94|
+--------------------+-----+



# Using DF

In [11]:

event_window = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

clickstream_w_error_flags = (spark_clickstream_df
                             .withColumn("error_flag", 
                                         f.max(
                                             f.when(f.col("event_type").like("%error%"), 1)
                                             .otherwise(0))
                                             .over(event_window))
                                             )
filtered_from_errors = clickstream_w_error_flags.filter(f.col('error_flag') != 1)

filtered_w_dup_flags = (filtered_from_errors
              .withColumn("duplicate_flag",
                    f.when(f.lag('event_page').over(event_window) == f.col('event_page'), 1)
                    .otherwise(0))
     )

deduplicated = filtered_w_dup_flags.filter(f.col('duplicate_flag') != 1)

deduplicated_routes = deduplicated.withColumn('sorted_route', 
                                              f.concat_ws("-", f.collect_list('event_page').over(event_window)))
max_routes = (deduplicated_routes
          .groupby('user_id', 'session_id')
          .agg(f.max('sorted_route').alias('route'))
         )


routes_df = (max_routes
                   .groupby('route').agg(f.count('*').alias('count'))
                   .orderBy(f.desc('count'))
                   .limit(30)
                  )

routes_df.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



In [24]:
# Convert DataFrame to RDD with required fields
clickstream_rdd = spark_clickstream_df.rdd.map(lambda row: 
    (row['user_id'], row['session_id'], row['event_type'], row['event_page'], row['timestamp']))

def filter_errors(events):
    error_flag = False
    filtered_events = []
    for event in events:
        user_id, session_id, event_type, event_page, timestamp = event
        if 'error' in event_type:
            error_flag = True
        if not error_flag:
            filtered_events.append(event)
    return filtered_events

filtered_clickstream_rdd = clickstream_rdd.groupBy(lambda x: (x[0], x[1])) \
    .flatMap(lambda x: filter_errors(sorted(x[1], key=lambda y: y[4])))

def remove_duplicates(events):
    unique_events = []
    last_page = None
    for event in events:
        user_id, session_id, event_type, event_page, timestamp = event
        if event_page != last_page:
            unique_events.append(event)
            last_page = event_page
    return unique_events

deduplicated_rdd = filtered_clickstream_rdd.groupBy(lambda x: (x[0], x[1])) \
    .flatMap(lambda x: remove_duplicates(sorted(x[1], key=lambda y: y[4])))

def generate_route(events):
    events = sorted(events, key=lambda x: x[4])  # Sort by timestamp
    route = '-'.join([event[3] for event in events])  # Concatenate event_pages
    user_id, session_id = events[0][0], events[0][1]
    return (route, 1)

routes_rdd = deduplicated_rdd.groupBy(lambda x: (x[0], x[1])) \
    .map(lambda x: generate_route(x[1])) \
    .reduceByKey(lambda a, b: a + b)

top_routes = routes_rdd.sortBy(lambda x: -x[1]).take(30)
top_routes_rdd = sc.parallelize(top_routes)
top_routes_df = top_routes_rdd.map(lambda x: Row(route=x[0], count=x[1])).toDF()

top_routes_df.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8185|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|   main-news-archive|  113|
|main-tariffs-inte...|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



In [25]:
routes_df = routes_df.withColumnRenamed("count", "df_count")
top_routes_df = top_routes_df.withColumnRenamed("count", "rdd_count")

comparison_df = routes_df.join(top_routes_df, on="route", how="inner")

mismatched_counts_df = comparison_df.filter(comparison_df["df_count"] != comparison_df["rdd_count"])

mismatched_counts_df.show(truncate=False)

+---------------------+--------+---------+
|route                |df_count|rdd_count|
+---------------------+--------+---------+
|main                 |8184    |8185     |
|main-tariffs-internet|114     |113      |
+---------------------+--------+---------+



In [30]:
import json

# Ensure the schema is correct by checking field names
routes_df.printSchema()

# Extract the top 10 routes as a list of Row objects
top_10_routes = routes_df.orderBy("df_count", ascending=False).limit(10).collect()

# Convert the results to a dictionary in the specified format
result_dict = {f"{row['route']}": row['df_count'] for i, row in enumerate(top_10_routes)}

# Save the dictionary to a JSON file
with open('result.json', 'w') as f:
    json.dump(result_dict, f, indent=4)

print("The top 10 most frequent routes have been saved to result.json.")


root
 |-- route: string (nullable = true)
 |-- df_count: long (nullable = false)



The top 10 most frequent routes have been saved to result.json.


In [31]:
!curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/maks88sgt/w6/2

0.9999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Correct main-online answer!
Correct main-vklad answer!
Correct main-rabota-archive answer!
